**Install Beautiful soup ,tqdm(optional) and selenium**

In [ ]:
!pip install Beautifulsoup4
!pip install tqdm

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm.notebook import tqdm
from multiprocessing import Pool
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

**The page contains drugs from 'A' to 'Z'**

In [2]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

**Extract page numbers for each letter**

In [ ]:
def urls_page_no(char):
  try:
    base_url = 'https://www.medindia.net/drug-price/' #The main url
    next_url = 'brand-index.asp?alpha='+char #The url to be added to go to a page level
    html_content = bs(requests.get(base_url+next_url).text,'html.parser') #Extract html
    last_url = html_content.find('ul',class_="pagination").find_all('li')[-1].find('a')['href']
    return re.findall(r'\d+',last_url) #Extract last page no.
  except:
    return ['1']

In [ ]:
page_nos = []            #Multiprocessing threads speeds up the process
with Pool(10) as p:
  results = p.map(urls_page_no,letters)
  page_nos.append(results)

In [ ]:
page_nos = [y for x in page_nos for y in x] #List of list to list

In [ ]:
page_nos = [int(i) for i in page_nos] 

**Extract the brands from the website**

In [ ]:
def brands_list(char,page_number):
  if int(page_number) >= 2:
    base_url = 'https://www.medindia.net/drug-price/'
    next_url = 'brand-index.asp?alpha='+char+'&page='+page_number
    table = bs(requests.get(base_url+next_url).text,'html.parser').find('table',class_="table-bordered table")
    return [drug.text for drug in table.find_all('a')]
  else:
    base_url = 'https://www.medindia.net/drug-price/'
    next_url = 'brand-index.asp?alpha='+char
    table = bs(requests.get(base_url+next_url).text,'html.parser').find('table',class_="table-bordered table")
    return [drug.text for drug in table.find_all('a')]

In [ ]:
drugs_list = []
for char,count in tqdm(zip(letters,range(len(letters)))):
  for idx in tqdm(range(0,page_nos[count])):
    drugs_list.append(brands_list(char,str(idx+1)))

In [ ]:
drugs_list = [y for x in drugs_list for y in x]

In [ ]:
df = pd.DataFrame(drugs_list) #Drug list on the main page

In [ ]:
df

,0
0,A - Cox - 450
1,A - Pandom - SR
2,A - Quin
3,A .G.T
4,A .S.A
...,...
80554,Zyven-OD (50mg)
80555,ZYX (200 mg)
80556,ZYX DT (100 mg)
80557,ZYX DT (50 mg)


In [ ]:
df.isna().sum()

0    0
dtype: int64

**Extract url list for all the drugs available on the site**

In [ ]:
def drug_urls(char,page_number):
  if int(page_number) == 1:
    base_url = 'https://www.medindia.net/drug-price/'
    next_url = 'brand-index.asp?alpha='+char
    html_content = bs(requests.get(base_url+next_url).text,'html.parser')
    last_url = html_content.find('table',class_="table-bordered table").find_all('a')
    return [line['href'] for line in last_url]
  else:
    base_url = 'https://www.medindia.net/drug-price/'
    next_url = 'brand-index.asp?alpha='+char+'&page='+page_number
    html_content = bs(requests.get(base_url+next_url).text,'html.parser')
    last_url = html_content.find('table',class_="table-bordered table").find_all('a')
    return [line['href'] for line in last_url]



In [ ]:
drugs_url_list = []
for char,count in tqdm(zip(letters,range(len(letters)))):
  for idx in tqdm(range(0,page_nos[count])):
    drugs_url_list.append(drug_urls(char,str(idx+1)))

In [ ]:
drugs_url_list = [y for x in drugs_url_list for y in x]

In [ ]:
len(drugs_url_list)

80559

In [ ]:
url_list = pd.DataFrame(drugs_url_list) #Full list of urls of the drugs

In [ ]:
url_list.to_csv('url_list.csv')

**Used bs with requests but failed since the site was blocking higher rate limit.**

In [19]:
def drug_info(url):
  html_content = bs(requests.get(url).text,'html.parser')
  drug_html_info = html_content.find('div',class_='bbox')
  tb_append = drug_html_info.find('a').extract().text
  info_data = [line.rstrip() for line in drug_html_info.text.strip().replace(u'\xa0',u' ').splitlines()]
  info_data.append(tb_append)
  return info_data
#Failed due to higher retrieval rate

In [3]:
drugs_url_list = pd.read_csv('url_list.csv',usecols=[1])

In [4]:
drugs_to_list = drugs_url_list.values.tolist()

In [5]:
drugs_to_list = [y for x in drugs_to_list for y in x]

In [6]:
len(drugs_to_list)

80559

In [11]:
drugs_to_list[0]

'https://www.medindia.net/drug-price/rifampicin/a-cox-450.htm'

In [ ]:
drug_info_list = []
for drugs in tqdm(drugs_to_list[0:500]):
  time.sleep(5)
  drug_info_list.append(drug_info(drugs))

**Used Opera webdriver with VPN to go around the block and used Google Cache service with randomized time sleep for emulating human like behaviour**

In [135]:
medindia_list = []
options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=/home/neerad/.config/opera' ) #Used VPN
driver = webdriver.Opera(executable_path='/home/neerad/Downloads/operadriver_linux64/operadriver',options=options) #Using Opera webdriver
for url in tqdm(drugs_to_list[0:500]): #First 500 drugs
    driver.get('http://webcache.googleusercontent.com/search?q=cache:'+url)
    html_content = driver.find_elements_by_xpath('//div[@class="bbox"]')
    medindia_list.append([x.text.splitlines() for x in html_content])
    time.sleep(random.randrange(60,100,1)) #Randomized between 1 minute to 100 sec since less than tht leads to blockage sooner or later.

In [136]:
medindia_list = [y for x in medindia_list for y in x]

In [137]:
len(medindia_list)

485

In [138]:
medindia_a_df = pd.DataFrame(medindia_list) ##Extracted only first 500 'A' lettered drugs

In [140]:
medindia_a_df.to_csv('drug_a_500_only.csv')